# Prerequisities

In [ ]:
#Install Packages 

!pip install baseball_scraper
!pip install baseball_id
!pip install pybaseball

In [14]:
# Import Packages

# Baseball Scraper
from baseball_scraper import baseball_reference
from baseball_scraper import espn
from baseball_scraper import fangraphs
from baseball_scraper import playerid_lookup
from baseball_scraper import statcast, statcast_pitcher, statcast_batter
from baseball_scraper import team_batting, batting_stats
from baseball_scraper import playerid_lookup

from pybaseball import playerid_reverse_lookup

#DateTime
from datetime import datetime
from datetime import date, timedelta


# Baseball ID
from baseball_id import Lookup

#Misc
import requests
import io
import bs4
import pandas as pd

In [8]:
# Important Global Variables

Today = date.today()
Tomorrow = date.today() + timedelta(days=1)
Yesterday = date.today() + timedelta(days=-1)

#MLB Players Dictionary, so it can loop and find values
MLB_Players = {}
#Let me try a different idea
# MLB_Players_ID_Dataframe
MLB_Players_ESPN_ID_Dict = {}
MLB_Players_MLB_ID_Dict = {}
Probable_Pitchers_and_Opponents_Dict = {}
#Add J. D. Martinez as exception because fuck this guy and his two first names
#If you're reading this, god damn you Julio Daniel's parents
MLB_Players_MLB_ID_Dict["JD Martinez"] = 502110

In [3]:
# Starting Pitcher Scraper
def Get_Probable_Pitchers(StartDate, EndDate):
  es = espn.ProbableStartersScraper (StartDate, EndDate)
  ProbablePitchers = es.scrape()
  return ProbablePitchers

# ProbablePitchers is a dataframe
#If we can run the names through the MLB_ID search as some sort of loop
#Then we can create IDs instantly for all these players
#if we can figure out a way to copy and paste that data,
#then we've got the next sets of pitchers coming out of the gates

def Probable_Pitcher_Opponents_Update():  
  ProbablePitchers = Get_Probable_Pitchers(Today, Today)
  Names_and_Opponents = ProbablePitchers[["Name","opponent"]]
  Probable_Pitchers_and_Opponents_Dict = dict(Names_and_Opponents.values)
  return
#What if I don't need a dictionary?

In [9]:
es = espn.ProbableStartersScraper (Today,Today)
ProbablePitchers = es.scrape()
ProbablePitchers

,Date,Name,espn_id,opponent
0,2021-07-27,Tyler Alexander,40360,MIN
1,2021-07-27,Kenta Maeda,34892,DET
2,2021-07-27,Austin Gomber,34897,LAA
3,2021-07-27,Jose Suarez,40981,COL
4,2021-07-27,Julio Urias,33223,SF
5,2021-07-27,Logan Webb,41216,LAD
6,2021-07-27,Lance McCullers Jr.,32764,SEA
7,2021-07-27,Chris Flexen,36067,HOU
8,2021-07-27,James Kaprielian,34927,SD
9,2021-07-27,Chris Paddack,35999,OAK


In [ ]:
def Add_To_MLB_Players_ID_Dataframe(SearchPlayerName):
  #PlayerFirstName,PlayerLastName=SearchPlayerName.split(" ")
 
 #Handled at GET_KEY level
  #Player_Name_Without_Spaces = PlayerFirstName + PlayerLastName
  #Once I get the key, track it to their name
  #MLB_Players[Player_Name_Without_Spaces] = Get_Key_MLBAM(SearchPlayerName,PlayerIndexNum)
  
  #PlayerIndexNum = 0
  #If it's a dictionary, I might not need to save the name without a space
  #MLB_Players_MLB_ID_Dict[SearchPlayerName] = Get_Key_MLBAM(SearchPlayerName,PlayerIndexNum)
  #need way to double check against identical names

  MLB_Players_MLB_ID_Dict[SearchPlayerName] = Get_Key_MLBAM(SearchPlayerName)
  #identical to above line, but with indexNum removed
  #Since the hope is that the first index will always be the player I'm looking for
  return

def Get_Key_MLBAM(PlayerName):
  #Keeping the Index Flexible right here, just in case
  PlayerIndexNum = 0
  PlayerFirstName,PlayerLastName=PlayerName.split(" ")
  ErrorMessage = "Try a Different Name"
  try:
      # We'll have to check if there's a duplicate name somehow
      Player_Dataframe = playerid_lookup(PlayerLastName,PlayerFirstName)
      #Trying to get the first result to be a player who played in 2021
      #Crashes the function for some reason, but it worked out of the function
      Player_Dataframe = Player_Dataframe.sort_values("mlb_played_last",na_position='last',ascending=False).reset_index(drop=True)
      #The above line throws an exception for some reason
      #Without that line, it works
      MLBKey_ID = Player_Dataframe.at[PlayerIndexNum,'key_mlbam']
  except:
      print(ErrorMessage)
      MLBKey_ID = "-2"
  return MLBKey_ID

In [38]:
PlayerName = "Fernando Tatis"
#Add_To_MLBID_Dictionary(PlayerName)
#Get_Key_MLBAM(PlayerName)
PlayerFirstName,PlayerLastName=PlayerName.split(" ")
Get_Key_MLBAM(PlayerName)

# Add_To_MLBID_Dictionary(PlayerName)
# MLB_Players_MLB_ID_Dict

Gathering player lookup table. This may take a moment.


665487

In [9]:
# Add to MLBID Dict Test

PlayerName = "Ohtani Shohei"
#Add_To_MLBID_Dictionary(PlayerName)
#Get_Key_MLBAM(PlayerName)
PlayerFirstName,PlayerLastName=PlayerName.split(" ")
Player_Dataframe = playerid_lookup(PlayerLastName,PlayerFirstName)
Player_Dataframe = Player_Dataframe.sort_values("mlb_played_last",na_position='last',ascending=False).reset_index(drop=True)
# Player_Dataframe.to_excel("PlayerKeys.xlsx")
Player_Dataframe
#MLB_Players_MLB_ID_Dict
#We want return: 665487

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last


In [39]:
fg_ids = [826, 5417, 210, 1101]
espn_ids = Get_Probable_Pitchers(Today, Today)

# find their names and ids from other data sources
data = playerid_reverse_lookup(list(espn_ids["espn_id"]), key_type='fangraphs')
data

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last


In [ ]:
# Player IDs key_mlbam
# Really Need an Automated List - Fixing at the Moment

#Pitchers
PaoloEspino = 502179
MaxScherzer = 453286
JonLester = 452657
BradHand = 543272
ShaneMcClanahan = 663556
MikeMinor = 501985
DanielHudson = 543339
ClaytonKershaw = 477132
MadisonBumgarner = 518516
MikeFoltynewicz = 592314
BradKeller = 641745
PatrickCorbin = 571578
KyleHendricks = 543294
JordanLyles = 543475
DallasKeuchel = 572971
ZachEflin = 621107
ChrisFlexen = 623167
JDMartinez = 502110
RyanYarbrough = 642232
ColeIrvin = 608344
ZachDuke = 435043
MartinPerez = 527048
EdwinJackson = 429719

#Hitters
VladGuerrero = 665489
FernandoTatis = 665487
TreaTurner = 607208
BryceHarper = 547180
AaronJudge = 592450
MikeTrout = 545361
JuanSoto = 665742
AdamFrazier = 624428
NickCastellanos = 592206
CedricMullins = 656775
YuliGurriel = 493329
JoseAltuve = 514888
BryanReynolds = 668804
JeanSegura = 516416
XanderBogaerts = 593428
DavidFletcher = 664058
MichaelBrantley = 488726
TimAnderson = 641313
JustinTurner = 457759
JesseWinker = 608385

#Shohei Ohtani
ShoheiOhtani = 660271

# Player Dictionary Functions

In [ ]:
MLB_Players_MLB_ID_Dict

#playerid_lookup(PlayerLastName,PlayerFirstName)

#MLB_Players_ESPN_ID_Dict

#Wainwright: 5403 ESPN 425749 MLB - absolutely no correlation
# Pretty sure nobody would ever have a reason to cross the two


In [2]:
Probable_Pitcher_Opponents_Update() #Doesn't work
  
ProbablePitchers = Get_Probable_Pitchers(Today, Today)
Names_and_Opponents = ProbablePitchers[["Name","opponent"]]
Names_and_Opponents

NameError: name 'Probable_Pitcher_Opponents_Update' is not defined

In [ ]:
#Trying to get the first result to be a player who played in 2021

PlayerName = "Fernando Tatis"
PlayerFirstName,PlayerLastName=PlayerName.split(" ")

Playerdf = playerid_lookup(PlayerLastName, PlayerFirstName)
#Playerdf_current = Playerdf[(Playerdf["mlb_played_last"] == 2021.0)]
#Playerdf_current
Playerdf = Playerdf.sort_values("mlb_played_last",na_position='last',ascending=False).reset_index(drop=True)
#Playerdf.reset_index(drop=True,inplace=True)
Playerdf
#Playerdf.reset_index(drop=True)

In [41]:
MLB_Players_MLB_ID_Dict["Marcus Stroman"]

KeyError: 'Marcus Stroman'

In [40]:
MLB_Players_ESPN_ID_Dict["Marcus Stroman"]

KeyError: 'Marcus Stroman'

In [ ]:
#make SearchPlayerName dynamic on a loop and we're golden
#Search for Player, add them and MLBID to dictionary
SearchPlayerName = "Madison Bumgarner"
PlayerIndexNum = 0

Get_Key_MLBAM(SearchPlayerName,PlayerIndexNum)

Add_To_MLBID_Dictionary(SearchPlayerName,PlayerIndexNum)

MLB_Players_MLB_ID_Dict

# Player Lookups

In [ ]:
#PlayerName = "Vladimir Guerrero"
#PlayerFirstName,PlayerLastName=PlayerName.split(" ")
#Playerdf = playerid_lookup(PlayerLastName,PlayerFirstName)

PlayerLastName = "Martinez"
Playerdf = playerid_lookup(PlayerLastName)
Playerdf_current = Playerdf[(Playerdf["mlb_played_last"] == 2021.0)]
Playerdf_current
#Playerdf_current.loc[[159]]
#(data['AB'] > 10) & (data['G'] > 40) & data['Team'].isin(opponents)].sort_index()

In [ ]:
# Baseball_Scraper PlayerID Lookup
# REDUNDANT BUT GOOD FOR TESTING AND REAPPROPRIATING

PlayerName = "J. D. Martinez"
ErrorMessage = "Try a Different Name"
PlayerFirstName,PlayerLastName=PlayerName.split(" ")

#Fuck it; I'm lazy
#PlayerFirstName = "Wasted"
#PlayerLastName = "Time"


#playerid_lookup(PlayerLastName,PlayerFirstName)

#Creates Copy and Paste Line for Players
#Visual Double Checks to Make Sure there's only one player
PlayerIndex = 0
#Actual Search Function
#I'm tired of seeing error messages and freaking out
try:
  Player_df = playerid_lookup(PlayerLastName,PlayerFirstName)
  print(Player_df)

#How can I get this to return key_mlbam when there is only one result?
#Just wanting the MLBKey_ID from this part
  MLBKey_ID = Player_df.at[PlayerIndex,'key_mlbam']
  print(PlayerFirstName+PlayerLastName,'=',MLBKey_ID)
except:
  print(ErrorMessage)

#playerid_lookup(PlayerLastName)


#The majority of names are a unique first and last name combo;
# could save time in creating database to check vs. MLB_Key
#If Multiple, user confirms -
#could do it just for the first time and save that choice
# I have it mostly automatic, which is nice I guess
# Just change the name, then copy and paste the results above
# Not sure if this will be necessary at any point in the future

In [ ]:
# Baseball_Scraper PlayerID Lookup
# Just Last Name - Started with J.D. Martinez
# Easier to search a million Martinez's than to figure out what JD stands for
# Turns out it's WAAAY easier to look up a player's page on MLB.com

PlayerLastName = "Guerrero"


#Data = playerid_lookup(PlayerLastName)
#Player_Possible_Key_Options_df[Player_Possible_Key_Options_df("last_played_MLB" = "2021.0")].sort_index()

# Pitcher Data

In [ ]:
Get_Probable_Pitchers(Today, Today)
ProbablePitchers

In [ ]:
#Dynamic Varibales We'll Need to Run The Statcast for Pitchers
#Need to convert dates to string for pitcher_stats
NumDaysInPast = 5
PitcherForStats = BradHand

In [ ]:
# Statcast Pitch Data by Pitcher

# Same Day results apparently not available
StartDate = str(Today+timedelta(days=-NumDaysInPast))
EndDate = str(Today)
#print(StartDate)

# formulatic testing - NLN
# Pitches come in reverse order i.e. the most recent pitch is pitch 0
#Pitcher_stats = statcast_pitcher('2021-07-24', '2021-07-25', DanielHudson)


#Pitcher_stats = statcast_pitcher()

#Inserting Dynamic Dates
Pitcher_stats = statcast_pitcher(StartDate, EndDate, PitcherForStats)


# Print Results
Pitcher_stats

#Limit Results to N pitches
#Pitches = 20
#Pitcher_stats.head(Pitches)


In [ ]:
# Statcast Pitch Data

>>> 
>>> data = statcast(start_dt='2017-06-24', end_dt='2017-06-27')
>>> data.head(5)

# Batter Data

In [ ]:
#Dynamic Varibales We'll Need to Run The Statcast
Batter_Num_Days_In_Past = 2
Batter_For_Stats = JDMartinez

In [ ]:
# Statcast Batter Data Fetch
# Copied from Pitcher Data

StartDate = str(Today+timedelta(days=-Batter_Num_Days_In_Past))
EndDate = str(Today)

Batter_stats = statcast_batter(StartDate,EndDate,Batter_For_Stats)

#Print The Results
Batter_stats

#Launch speed and launch angle give good estimates of where a batter is hitting the balls
#Located after hit_distance
#One of the columns: Estimated BA using Speedangle? Sounds promising
#Located after fielder IDs and Release-Y


In [ ]:
#I want an easy way to pick and choose data
#Maybe concentrate on 3 true outcomes? - Would bias toward players who hit homers
Batter_Speed_Angle = Batter_stats[["description", "des", "estimated_ba_using_speedangle", "launch_speed", "launch_angle","events","pitch_name","pitch_number"]]
Batter_Speed_Angle

In [ ]:
# ????

player_id = Lookup.from_names(['Khris Davis']).iloc[0].fg_id
fangraphs.Scraper.instances()
fg = fangraphs.Scraper("Steamer (RoS)")
df = fg.scrape(player_id, scrape_as=fangraphs.ScrapeType.HITTER)
df.columns

In [ ]:
# Batter Scraper

data = batting_stats(2010)
# data[data['Team'] == 'WSN'] 
data

# Team Lookup

In [ ]:
for i in ProbablePitchers['opponent'].unique():
  print(i)

In [ ]:
tss = baseball_reference.TeamSummaryScraper()
# tss
# tss.scrape(2021)
# try:
#   tss.scrape(2021)
# except Exception as e:
#   print('test', e)
# df.columns

In [ ]:
opponents = []
for i in ProbablePitchers["opponent"].unique():
  if (i == "KC"):
    opponents.append("KCR")
  elif (i == "SD"):
    opponents.append("SDP")
  elif (i == "SF"):
    opponents.append("SFG")
  elif (i == "TB"):
    opponents.append("TBR")
  elif (i == "WSH"):
    opponents.append("WSN")
  else:
    opponents.append(i)

print(opponents)
data[(data['AB'] > 10) & (data['G'] > 40) & data['Team'].isin(opponents)].sort_index()
# data["Team"].unique()
# data[data['Team'] == '- - -']

In [21]:
batting_stats(2021)

IndexError: list index out of range

In [ ]:
test = requests.get('https://www.baseball-reference.com/previews/2021/KCA202107250.shtml')
page = bs4.BeautifulSoup(test.content)

table = page.find('table', id='batters_vs_pitchers_DET')

# Depreciated

In [ ]:
# Add Probable Pitchers to ESPNID Dictionary
def Probable_Pitcher_ESPNID_Update():  
  ProbablePitchers = Get_Probable_Pitchers(Today, Today)
  Names_and_ESPNIDs = ProbablePitchers[["Name","espn_id"]]
  MLB_Players_ESPN_ID_Dict = dict(Names_and_ESPNIDs.values)
  return

In [ ]:
def Add_To_ESPNID_Dictionary(SearchPlayerName,PlayerIndexNum):
  PlayerFirstName,PlayerLastName=SearchPlayerName.split(" ")
  Player_Name_Without_Spaces = PlayerFirstName + PlayerLastName
  #Once I get the key, track it to their name
  MLB_Players_ESPN_ID_Dict[Player_Name_Without_Spaces] = Get_Key_MLBAM(SearchPlayerName,PlayerIndexNum)
  return
#Need to take names from this dictionary (or probable pitchers dictionary)
#Take names and grab MLB IDs, which has statcast data
#Could eliminate duplicate names with "Last Played 2021 in lookup"
#which I don't know how to do
#Once I pare down a name to a single MLBID, use the add to Player Dictionary function

In [ ]:
Probable_Pitcher_ESPNID_Update()

ProbablePitchers = Get_Probable_Pitchers(Today,Today)
Names_and_ESPNIDs = ProbablePitchers[["Name","espn_id"]]
MLB_Players_ESPN_ID_Dict = dict(Names_and_ESPNIDs.values)

MLB_Players_ESPN_ID_Dict

#We don't actually need the ESPN IDs
#ESPN doesn't have any data we want at the moment